# Simulates the Tritium Breeding Ratio as a function of lithium enrichment

- Lithium isotopes have a natural abundances of 7.59% Li6 and 92.41% Li7.
- However, due to the different cross sections of the two isotopes it is often beneficial (for TBR) to enrich the Li6 content.
- This example makes a simple model with a lithium containing blanket and a controllable enrichment.
- Simulations are performed for several different Li6 enrichments and the TBR as a function of enrichment is found.

This section creates a neutronics model for the provided enrichment value, runs the simulation and returns the TBR value obtained.

In [ ]:
import openmc
import plotly.graph_objects as go
from tqdm import tqdm


def make_materials_geometry_tallies(enrichment):
    """Makes a neutronics model of a blanket and simulates the TBR value.

    Arguments:
        enrichment (float): the enrichment percentage of Li6 in the breeder material
    
    Returns:
        resutsl (dict): simulation tally results for TBR along with the standard deviation and enrichment
    """

    # MATERIALS

    breeder_material = openmc.Material(1, "PbLi")  # Pb84.2Li15.8
    breeder_material.add_element('Pb', 84.2, percent_type='ao')
    breeder_material.add_element('Li', 15.8, percent_type='ao', enrichment=enrichment, enrichment_target='Li6', enrichment_type='ao')
    breeder_material.set_density('atom/b-cm', 3.2720171e-2)  # around 11 g/cm3


    steel = openmc.Material(name='steel')
    steel.set_density('g/cm3', 7.75)
    steel.add_element('Fe', 0.95, percent_type='wo')
    steel.add_element('C', 0.05, percent_type='wo')

    mats = openmc.Materials([breeder_material, steel])


    # GEOMETRY

    # surfaces
    vessel_inner = openmc.Sphere(r=500)
    first_wall_outer_surface = openmc.Sphere(r=510)
    breeder_blanket_outer_surface = openmc.Sphere(r=610, boundary_type='vacuum')


    # cells
    inner_vessel_region = -vessel_inner
    inner_vessel_cell = openmc.Cell(region=inner_vessel_region)

    first_wall_region = -first_wall_outer_surface & +vessel_inner
    first_wall_cell = openmc.Cell(region=first_wall_region)
    first_wall_cell.fill = steel

    breeder_blanket_region = +first_wall_outer_surface & -breeder_blanket_outer_surface
    breeder_blanket_cell = openmc.Cell(region=breeder_blanket_region)
    breeder_blanket_cell.fill = breeder_material

    universe = openmc.Universe(cells=[inner_vessel_cell, first_wall_cell, breeder_blanket_cell])
    geom = openmc.Geometry(universe)


    # SIMULATION SETTINGS
    sett = openmc.Settings()
    sett.batches = 10  # this is minimum number of batches that will be run
    sett.trigger_active = True
    sett.trigger_max_batches =  100  # this is maximum number of batches that will be run
    sett.inactive = 0
    sett.particles = 1000
    sett.run_mode = 'fixed source'

    source = openmc.Source()
    source.space = openmc.stats.Point((0, 0, 0))
    source.angle = openmc.stats.Isotropic()
    source.energy = openmc.stats.Discrete([14e6], [1])
    sett.source = source

    # TALLIES

    tallies = openmc.Tallies()

    cell_filter = openmc.CellFilter(breeder_blanket_cell)
    tbr_tally = openmc.Tally(name='TBR')
    tbr_tally.filters = [cell_filter]
    tbr_tally.scores = ['(n,Xt)']  # Where X is a wildcard character, this catches any tritium production
    tbr_tally.triggers = [openmc.Trigger(trigger_type='std_dev', threshold=0.01)]  # This stops the simulation if the threshold is meet
    tallies.append(tbr_tally)

    # RUN OPENMC
    model = openmc.model.Model(geom, mats, sett, tallies)
    !rm *.h5
    sp_filename = model.run(output=False)  # runs with reduced amount of output printing

    # OPEN OUPUT FILE
    sp = openmc.StatePoint(sp_filename)

    tbr_tally = sp.get_tally(name='TBR')

    df = tbr_tally.get_pandas_dataframe()

    tbr_tally_result = df['mean'].sum()
    tbr_tally_std_dev = df['std. dev.'].sum()

    return {'enrichment': enrichment,
            'tbr_tally_result': tbr_tally_result,
            'tbr_tally_std_dev': tbr_tally_std_dev}

Obtains TBR results for a range of different blanket enrichments.

In [ ]:
results = []
for enrichment in tqdm([0, 25, 50, 75, 100]):  # percentage enrichment from 0% Li6 to 100% Li6
    results.append(make_materials_geometry_tallies(enrichment))

results

Plots a figure showing TBR as a function of Li6 enrichment.

In [ ]:
fig = go.Figure()

# PLOTS RESULTS
fig.add_trace(
    go.Scatter(
        x=[entry['enrichment'] for entry in results],
        y=[entry['tbr_tally_result'] for entry in results],
        mode='lines',
        error_y={'array': [entry['tbr_tally_std_dev'] for entry in results]},
        )
)

fig.update_layout(
      xaxis={'title': 'Li6 enrichment (%)'},
      yaxis={'title': 'TBR'}
)

fig.show()

**Learning Outcomes for Task 5**
- Finding TBR with OpenMC.
- Simple methods of increasing the TBR using lithium enrichment.